In [37]:
from treelib import Node, Tree

#create tree object from an annotation file
#Sample Annotation:
#
#
# What is the way things are?
# >###Q_1###
# >>###Answer_1’###
# >>###Question_2###
# >>>###Answer_2’###
# >>>###Answer_2’’###
# >>###Question_3###
# >>>###Answer_3###
# >>>###Question_4###
# >>>>###Answer_4###
# >>###Answer_1’’###
#
def createTreeFromFile(filename):
    tree = Tree()
    with open (filename, "r") as myfile:
        data=myfile.readlines()
    level_dict = dict()
    node_no=0
    tree.create_node(getText(data[0]), node_no)
    level_dict[0] = node_no
    for node in data[1:]:
        node_no = node_no+1
        level = getLevel(node)
        text = getText(node)
        node_type = getType(node)
        tree.create_node(text,node_no,parent=level_dict[level-1])
        level_dict[level] = node_no

    tree.show(key=lambda x: x.identifier)

    return tree

In [38]:
import re
def getText(string):
    p = re.compile(">*(.*)\n")
    return p.match(string).group(1)

def getLevel(string):
    p = re.compile(">*")
    return p.match(string).span()[1]-p.match(string).span()[0]

def getType(string):
    p = re.compile(">*(A|Q*)")
    return p.match(string).group(1)

In [39]:
createTreeFromFile("sample.txt")

What is the way things are?
└── ###Q_1###
    ├── ###Answer_1’###
    ├── ###Question_2###
    │   ├── ###Answer_2’###
    │   └── ###Answer_2’’###
    ├── ###Question_3###
    │   ├── ###Answer_3###
    │   └── ###Question_4###
    │       └── ###Answer_4###
    └── ###Answer_1’’###

